##비용 최소화하기-경사 하강법(Gradient Descent) 소개

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

np.random.seed(0)
#y = 4*x+6를 근사(w1=4,w0=6). 임의의 값은 노이즈를 위해 만듦.
X = 2*np.random.rand(100,1)
y = 6 + 4*X + np.random.rand(100,1)

#X,y 데이터 세트 산점도로 시각화
plt.scatter(X,y)

###get_weight_update 함수  
w1과 w0을 업데이트 할 w1_update,w0_update 를 반환

In [ ]:
def get_weight_updates(w1,w0,X,y,learning_rate=0.01):
    N = len(y)
    #먼저 w1_update,w0_update를 각각 w1,w0의 shape와 동일한 크기를 가진 0값으로 초기화
    w1_update = np.zeros_like(w1)
    w0_upate = np.zeros_like(w0)
    #예측 배열 계산하고 예측과 실제 값의 차이 계산
    y_pred = np.dot(X,w1.T) + w0
    diff = y-y_pred
    
    #w0_update를 dot 행렬 연산으로 구하기 위해 모두 1값을 가진 행렬 생성
    w0_factors = np.ones((N,1))
    #w1과 w0을 업데이트할 w1_update와 w0_update 계산
    w1_update = -(2/N)*learning_rate*(np.dot(X.T,diff))
    w0_update = -(2/N)*learning_rate*(np.dot(w0_factors.T,diff))

    return w1_update,w0_update

###gradient_descent_steps 함수  
경사 하강 방식으로 반복적으로 수행하여 w1과 w0를 업데이트 한다.

In [ ]:
#입력 인자 iters 로 주어진 횟수만큼 반복적으로 w1과 w0를 업데이트 적용함.
def gradient_descent_steps(X,y,iters=1000):
    #w0와 w1을 모두 0으로 초기화
    w0 = np.zeros((1,1))
    w1 = np.zeros((1,1))

    #인자로 주어진 iters 만큼 반복적으로 get_weight_updates() 호출해 w1,w0 업데이트 수행
    for ind in range(iters):
        w1_update, w0_update = get_weight_updates(w1,w0,X,y,learning_rate=0.01)
        w1 = w1 - w1_update
        w0 = w0 - w0_update
    return w1,w0

###get_cost 함수
예측값과 실제값의 RSS 차이를 계산한다.

In [ ]:
#비용 함수를 정의해보자
def get_cost(y,y_pred):
    N = len(y)
    cost =np.sum(np.square(y-y_pred))/N
    return cost

In [ ]:
w1,w0 = gradient_descent_steps(X,y,iters=1000)
print("w1:{0:.3f} w0:{1:.3f}".format(w1[0,0],w0[0,0]))
y_pred = w1[0,0]*X + w0
print("Gradient Descent Total Cost:{0:.4f}".format(get_cost(y,y_pred)))

실제 선형식인 y=4X + 6 과 유사하게 w1 = 4.028, w0 = 6.490 가 도출되었다. 예측 오류 비용은 약 0.0775 가 나온다. 앞에서 구한 y_pred에 기반해 회귀선을 그려 보겠다.

In [ ]:
plt.scatter(X,y)
plt.plot(X,y_pred)

##(미니 배치) 확률적 경사 하강법  
일부 데이터만 이용해 W가 업데이트되는 값을 계산한다.

### stochastic_gradient_descent_steps 함수  
전체 X,y 데이터에서 랜덤하게 batch_size만큼 데이터를 추출해 이를 기반으로 w1_update,w0_update를 계산한다.

In [ ]:
def stochastic_gradient_descent_steps(X,y,batch_size=10,iters=1000):
    w0 = np.zeros((1,1))
    w1 = np.zeros((1,1))

    prev_cost = 100000
    iter_index = 0

    for ind in range(iters):
        np.random.seed(ind)
        #전체 X,y 데이터에서 랜덤하게 batch_size만큼 데이터를 추출해 sample_X,sample_y로 저장
        stochastic_random_index = np.random.permutation(X.shape[0])
        sample_X = X[stochastic_random_index[0:batch_size]]
        sample_y = y[stochastic_random_index[0:batch_size]]
        #랜덤하게 batch_size만큼 추출된 데이터 기반으로 w1_update,w0_update 계산 후 업데이트
        w1_update,w0_update = get_weight_updates(w1,w0,sample_X,sample_y,learning_rate=0.01)
        w1 = w1 - w1_update
        w0 = w0 - w0_update
        
        return w1,w0

In [ ]:
X.shape[0]

In [ ]:
index = np.random.permutation(X.shape[0])
index

In [ ]:
X_index = index[0:10]
X_index    #X 추출될 10개의 숫자의 인덱스

In [ ]:
X[X_index]   #X_index를 가진 10개의 숫자가 추출됨.

In [ ]:
w1,w0 = stochastic_gradient_descent_steps(X,y,iters=1000)
print("w1:",round(w1[0,0],3),"w0:",round(w0[0,0],3))
y_pred = w1[0,0]*X +w0
print("Stochastic Gradient Descent Total Cost:{0:.4f}".format(get_cost(y,y_pred)))

## 다항 회귀와 과대적합/과소적합 이해

[x_1,x_2] = [0,1]  
[x_1.x_2] = [2,3]

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
import numpy as np

#다항식으로 변환한 단항식 생성, [[0,1],[2,3]]의 2x2 행렬 생성
X = np.arange(4).reshape(2,2)
print("일차 단항식 계수 피처:\n",X)

#degree =2 인 2차 다항식으로 변환하기 위해 PolynomialFeatures를 이용해 변환
poly = PolynomialFeatures(degree=2)
poly.fit(X)
poly_ftr = poly.transform(X)
print('변환된 2차 다항식 계수 피처:\n',poly_ftr)

3차 다항 회귀의 결정함수식은 다음과 같이 y = 1+2*x_1 + 3*x_1^2 + 4*x_2^3 로 설정하고 이를 위한 함수 polynomial_func() 함수를 만든다.

In [ ]:
def polynomial_func(X):
    y = 1+2*X[:,0] + 3*X[:,0]**2 + 4*X[:,1]**3
    return y
X = np.arange(4).reshape(2,2)
print('일차 단항식 계수 feature: \n',X)
y = polynomial_func(X)
print('삼차 다항식 결정값:\n',y)

In [ ]:
X[:,0]

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
#3차 다항식 변환
poly_ftr = PolynomialFeatures(degree=3).fit_transform(X)
print('3차 다항식 계수 feaeture:\n',poly_ftr)

#Linear Regression에 3차 다항식 계수 feature와 3차 다항식결정값으로 학습 후 회귀 계수 확인
model = LinearRegression()
model.fit(poly_ftr,y)
print('Polynomial 회귀 계수\n',np.round(model.coef_,2))
print('Polynomial 회귀 shape:',model.coef_.shape)

[0.   0.18 0.18 0.36 0.54 0.72 0.72 1.08 1.62 2.34] 는 1+2*x_1 + 3*x_1^2 + 4*x_2^3 의 계수 값인 [1,2,0,3,0,0,0,0,0,4] 와는 차이가 있지만 다항 회귀로 근사하고 있음을 알 수 있다. 이처럼 사이킷런은 PolynomialFeatures로 피처를 변환한 후에 LinearRegression 클래스로 다항 회귀를 구현한다.

바로 앞 예제와 같이 피처 변환과 선형 회귀 적용을 각각 별도로 하는 것보다는 사이킷런의 Pipeline객체를 이용해 한 번에 다항 회귀를 구현하는 것이 코드를 더 명료하게 작성하는 방법이다.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
import numpy as np

def polynomial_func(X):
    y = 1+2*X[:,0] + 3*X[:,0]**2 + 4*X[:,1]**3
    return y

#Pipeline 객체로 Streamline하게 Polynomial Feature 변환과 Linear Regression을 연결
model = Pipeline([('poly',PolynomialFeatures(degree=3)),('linear',LinearRegression())])
X = np.arange(4).reshape(2,2)
y = polynomial_func(X)

model = model.fit(X,y)

print('Polynomial 회귀 계수\n',np.round(model.named_steps['linear'].coef_,2))